# 1. Setup and Imports

In [1]:
import os
import sys
import pandas as pd

# Add src/ to path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', 'src')))

from utils import log_step

# 2. Load Raw Data (from CSV or Oracle)

In [ ]:
log_step("Loading scraped reviews...")
df_raw = pd.read_csv("../data/bank_reviews_raw.csv")
df_raw = df_raw.rename(columns={"date": "review_date"})
df_raw.head()

# 3. Remove Duplicates and Nulls

In [ ]:
log_step("Removing duplicates and missing values...")
initial_shape = df_raw.shape

df_cleaned = df_raw.drop_duplicates(subset=["review", "app_name"])
df_cleaned.dropna(subset=["review", "rating", "review_date"], inplace=True)

log_step(f"Removed {initial_shape[0] - df_cleaned.shape[0]} rows")

# 4. Normalize Date Format

In [ ]:
log_step("Normalizing date format...")
df_cleaned["review_date"] = pd.to_datetime(df_cleaned["review_date"]).dt.date  # YYYY-MM-DD

In [ ]:
# Import translation utility
from translation_utils import translate_reviews

# Translate reviews
df_cleaned = translate_reviews(df_cleaned)

# Display sample translations
print("\nSample translations:")
sample_translations = df_cleaned[['review', 'review_translated']].head(5)
display(sample_translations)

# 5. Validate Data

In [ ]:
log_step("Summary of cleaned data:")
print(df_cleaned["app_name"].value_counts())
print(df_cleaned["rating"].value_counts())

# 6. Save Cleaned Data

In [ ]:
log_step("Saving cleaned data to CSV...")
df_cleaned.to_csv("../data/bank_reviews_cleaned.csv", index=False)

# Save DataFrame to Oracle

In [ ]:
from database import get_engine, insert_dataframe

engine = get_engine()

insert_dataframe(df_cleaned, table_name='bank_reviews_cleaned', engine=engine)